***vis_completables_progress* function**:
* Input: players_info dictionary 
* Displays bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game

In [1]:
def vis_completables_progress (players_info):
    # find full list of completables
    completables=set()
    for player in players_info.keys():
        if players_info[player]["completables_progress"] != []:
            completables = completables.union(players_info[player]["completables_progress"].keys())
    completables = list(completables)

    # progress of each player per completable
    players_progress = []
    for player in players_info.keys():
        if players_info[player]["completables_progress"] != []:
            player_progress = []
            for c in completables:
                if c in players_info[player]["completables_progress"].keys():
                    height = players_info[player]["completables_progress"][c]
                else:
                    height = 0
                player_progress.append(height)
            players_progress.append(player_progress)
            
            
    # Bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game.

    n_groups = len(completables)

    fig, ax = plt.subplots()
    index = np.arange(n_groups)
    bar_width = 0.2
    opacity = 0.8

    for i in range(len(players_progress)):
        plt.bar(index+i*bar_width, players_progress[i], bar_width, 
                label=list(players_info.keys())[i], 
                tick_label=completables)

    plt.legend()
    plt.xlabel("Completable")
    plt.ylabel("Progress")
    plt.ylim((0,1.05))
    plt.title("Progress of players per completable")
    plt.show()